In [36]:
import xarray
import rioxarray # to open and download remote raster data
from rioxarray.merge import merge_arrays
import numpy as np
from stl import mesh

In [37]:
# Load merged tiles
tif_path = r"C:\Users\ngsim\Downloads\241218_30m_SingleSkiTrack_Cities1900.tif"
MergedTiles = rioxarray.open_rasterio(tif_path) # open tif as xarray DataArray
MergedTiles.shape # print DataArray shape

C:\Users\ngsim\UserApps_SN\Anaconda3\lib\site-packages\pyproj\crs\_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


(1, 1267, 1367)

In [38]:
def add_mask_edges(mask):
        '''
        Code inspired by ChatGPT, but with a different ultimate method to find edges by looping through every point
        
        Update mask to include points adjacent to non-excluded points.
    
        Parameters:
            mask (numpy.ndarray): A boolean mask where True indicates valid points and False indicates excluded points.
    
        Returns:
            numpy.ndarray: A boolean mask where True indicates points adjacent to valid points.'''
        
        # Create a padded version of the mask to handle edge cases
        padded_mask = np.pad(mask, pad_width=1, mode='constant', constant_values=False) # pads each dimension of the array with 1 row/column. i.e. (1, 380, 594) becomes (3, 382, 596)
        print(f"Padded mask shape is {padded_mask.shape}")
        
        # Initialize the new mask
        mask_plus_edges = np.zeros_like(mask, dtype=bool) # has same size as original mask, all set to false
        
        # Check each point in mask
        for i in range(mask.shape[1]): # for rows
            for j in range(mask.shape[2]): # and columns
                
                # check in all directions to see if there is a True value for mask
                for i_offset in [-1, 0, 1]:
                    for j_offset in [-1, 0, 1]:
                        
                        row = i + i_offset + 1 # +1 is required to account for the extra left/top column/row in the padded array
                        col = j + j_offset + 1
                        
                        if padded_mask[1][row][col]: # if adjacent or central cell is true in mask (padded mask allows original mask edges to be evaluated without error)
                            mask_plus_edges[0][i][j] = True # then central cell should be included in the new mask
                            
        return mask_plus_edges

In [39]:
# Process tif image
image = MergedTiles.values # convert xarray DataArray into numpy array

# Mask out the flat areas with the exclude_value (not sure if the mask is working properly)
exclude_value = image.min() # excluding the minimum value removes areas outside the desired mountainous region that are either assigned 0 or -9999 depending on how the tif was exported
mask = image != exclude_value  # True where values are valid
mask = add_mask_edges(mask) # function to add one column/row of True values around the original mask so that sides can be calculated for the stl file

valid_image = np.where(image > 1000, image, 1000)  # Replace excluded values with base model elevation
valid_image.min() # print 

Padded mask shape is (3, 1269, 1369)


1000.0

In [40]:
# Normalize pixel values to a height range
pixel_size = 15 # tif pixel size in meters
height_map = valid_image/pixel_size # scale height by pixel size (in meters)
height_map *= mask  # Apply mask to keep only valid regions
rows, cols = height_map[0].shape # Get the dimensions of the image
mask = mask[0] # reduce mask array to 2D
height_map = height_map[0]  # reduce height_map array to 2D
print(height_map.min())
print(height_map.max())

0.0
229.61441


In [10]:
# optional code to split map into 5 vertical pieces to reduce memory requirements

split_map = False

if split_map:
    row_segments = round(rows/5) # number of rows in each map segment
    map_segment1 = height_map[:row_segments] # slice out first set of rows
    map_segment2 = height_map[row_segments:row_segments*2] # slice out second set of rows, etc
    map_segment3 = height_map[row_segments*2:row_segments*3]
    map_segment4 = height_map[row_segments*3:row_segments*4]
    map_segment5 = height_map[row_segments*4:]
    rows = row_segments # update rows for subsequent vertices and faces loops to run through each map segment instead of the entire map
    height_map = map_segment1 # udpate height_map for subsequent loops with desired map segment to process
    map_segment1.shape # print the map segment rows and columns

In [41]:
# Generate mesh and export stl file (code from ChatGPT with minor edits to improve processing (changing vartices and faces arrays to int))

# Create vertices (only for valid points)
vertices = []
vertex_map = -np.ones((rows, cols), dtype=int)  # Map to track valid vertices
vertex_index = 0

for i in range(rows):
    for j in range(cols):
        if mask[i, j]:  # Only add vertices for valid points
            vertices.append([i, j, height_map[i, j]])
            vertex_map[i, j] = vertex_index
            vertex_index += 1

vertices = np.array(vertices).astype(int) # convert vertices list to numpy array as integer to reduce processing time
print("Vertices created")


# Create faces (only for valid grid cells)
faces = []
for i in range(rows - 1):
    for j in range(cols - 1):
        if mask[i, j] and mask[i, j + 1] and mask[i + 1, j]:  # Valid first triangle
            top_left = vertex_map[i, j]
            top_right = vertex_map[i, j + 1]
            bottom_left = vertex_map[i + 1, j]
            faces.append([top_left, bottom_left, top_right])

        if mask[i + 1, j] and mask[i, j + 1] and mask[i + 1, j + 1]:  # Valid second triangle
            top_right = vertex_map[i, j + 1]
            bottom_left = vertex_map[i + 1, j]
            bottom_right = vertex_map[i + 1, j + 1]
            faces.append([top_right, bottom_left, bottom_right])

faces = np.array(faces).astype(int) # convert faces list to numpy array as integer to reduce processing time
print("Faces created")


# Create the mesh
terrain_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

for i, face in enumerate(faces):

    # Print percent complete (the large number of print statements filled up memory)
    #print(i/len(faces)*100)

    for j in range(3):
        terrain_mesh.vectors[i][j] = vertices[face[j], :]
print("Mesh created")
        
    
# Save as STL
stl_path = r"C:\Users\ngsim\Downloads\241219_30m_sides.stl"
terrain_mesh.save(stl_path)
print(f"STL saved to {stl_path}")

Vertices created
Faces created
Mesh created
STL saved to C:\Users\ngsim\Downloads\241219_30m_sides.stl
